# Generate charts for trading analysis using Historical Pricing API

## Overview

Technical Analysis for intraday trading is one of the basic approaches to help you choose the planning of ventures. Basically, users can use the intraday or end-of-day pricing data to generate charts such as the CandleStick. And then, they can review the stock price' trend and make decisions using their own mehtod or algorithm. Typically, LSEG Workspace and Eikon Desktop users also provide access to this information, and it also provides a user interface to display multiple types of Charts. 

However, there is a specific requirement from a customer who did not use Eikon or LSEG Workspace. They need to build their own stand-alone application or generate the charts on their application using data from LSEG. The user is looking for the source of data or service to provide the intraday data for them in the REST API format. So they can integrate the service with their app. 

To support the requirement client can use REST API provided by [Data Platform](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis) that basically provides simple web-based API access to a broad range of content provided by LSEG. It can retrieve data such as News, ESG, Symbology, Streaming Price, and Historical Pricing. Users can get the intraday from the Historical Pricing service and then pass the data to any libraries that provided functionality to generate Charts. To help API users access content easier, LSEG also provides the LSEG Data Platform Libraries that provide a set of uniform interfaces providing the developer access to the Data Platform. We are currently providing an LSEG Data Platform Libraries for Python, TypeScript and .NET users. A general user and data scientists can leverage the library's functionality to retrieve data from the service, which basically provides the original response message from the service in JSON tabular format. The LSEG Data Library for Python will then convert the JSON tabular format to the pandas dataframe so that the user does not need to handle a JSON message and convert it manually. The user can use the dataframe with libraries such as mathplotlib and mplfinance to display the Charts.


In this example, I will show you how to use the LSEG Data Library for Python, request the intraday data from the Historical Pricing service, and then generate basic charts such as the Candle Stick charts. Basically, the [Japanese candlestick chart](https://www.investopedia.com/trading/candlestick-charting-what-is-it/) commonly used to illustrate movements in the price of a financial instrument over time. It's popular in finance, and some technical analysis strategies use them to make trading decisions, depending on the candles' shape, color, and position.

## Prerequisites

* Follow instructions from [Quick Start Guide](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/quick-start) to install LSEG Data Library for python.

    ```bash
    > pip install lseg-data
    ```

* You must have an RDP Account or LSEG Workspace desktop application with permission to request data using Historical Pricing API.

* Ensure that you have the following additional python libraries.

  ```bash
  matplotlib, mplfinance, seaborn, ipympl
  ```
  


## Getting Started using LSEG Data Library for python

The library supports the configuration file (lseg-data.config.json). The configuaration file can be modified to connect to the desktop session or data platform session. 

Set the __"app-key"__ property and set the default session to __"desktop.workspace"__ to connect to the desktop session.

```bash
    "sessions": {
        "default": "desktop.workspace",
        ...
        "desktop": {
            "workspace": {
                "app-key": "<APP KEY>"
            }
        }
    }
```
Set the __"app-key"__, __"username"__, and __"password"__ properties and set the default session to __"platform.ldp"__ to connect to the data platform session.

```bash
    "sessions": {
        "default": "platform.ldp",
        "platform": {
            "ldp": {
                "app-key": "YOUR APP KEY GOES HERE!",
                "username": "YOUR LDP LOGIN OR MACHINE GOES HERE!",
                "password": "YOUR LDP PASSWORD GOES HERE!"
            },
           ...
    }
```

### Open the Session

Next step, we will import __[lseg.data](https://pypi.org/project/lseg-data/)__ libraries, set the configuration's location, and open the default session. 

In [ ]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "./"

import lseg.data as ld
from lseg.data.content import historical_pricing
from lseg.data.content.historical_pricing import Intervals
from lseg.data.content.historical_pricing import Adjustments
from lseg.data.content.historical_pricing import MarketSession


global ricName

ld.open_session()

### Retreive Time Series data

After the session state is Open, we will use the __historical_pricing.summaries.Definition__ interface to retrieve time series pricing Interday summaries data(i.e., bar data).

```python
Definition(universe: 'StrStrings', interval: Union[str, lseg.data.content._intervals.Intervals] = None, start: 'OptDateTime' = None, end: 'OptDateTime' = None, adjustments: 'OptAdjustments' = None, sessions: 'OptMarketSession' = None, count: 'OptInt' = None, fields: 'StrStrings' = None, extended_params: 'ExtendedParams' = None) -> None
```

Actually, the implementation of this function will send HTTP GET requests to the historical pricing endpoints.

And the following details are possible values for interval and adjustment arguments.

Supported __intervals__:
- __Interday intervals__: Intervals.DAILY, Intervals.ONE_DAY, Intervals.WEEKLY, Intervals.MONTHLY, Intervals.THREE_MONTHS, Intervals.TWELVE_MONTHS, Intervals.YEARLY.

- __Intraday intervals__: Intervals.FIVE_MINUTES, Intervals.HOURLY, Intervals.MINUTE, Intervals.SIXTY_MINUTES, Intervals.TEN_MINUTES, Intervals.THIRTY_MINUTES


Supported value for __adjustments__:

- __Adjustments.UNADJUSTED__ - Not apply both exchange/manual corrections and CORAX
- __Adjustments.EXCHANGE_CORRECTION__ - Apply exchange correction adjustment to historical pricing
- __Adjustments.MANUAL_CORRECTION__ - Apply manual correction adjustment to historical pricing, i.e. annotations made by content analysts
- __Adjustments.CCH__ - Apply Capital Change adjustment to historical Pricing due to Corporate Actions e.g. stock split
- __Adjustments.CRE__ - Apply Currency Redenomination adjustment when there is redenomination of currency
- __Adjustments.RPO__ - Apply Reuters Price Only adjustment to adjust historical price only not volume
- __Adjustments.RTS__ - Apply Reuters TimeSeries adjustment to adjust both historical price and volume
- __Adjustments.QUALIFIERS__ - Apply price or volume adjustment to historical pricing according to trade/quote qualifier summarization actions

You can pass an array of these values to the function like the below sample codes.

```python
 adjustments = [
        Adjustments.EXCHANGE_CORRECTION,
        Adjustments.MANUAL_CORRECTION
    ]
```

The adjustments are a query parameter that tells the system whether to apply or not apply CORAX (Corporate Actions) events or exchange/manual corrections or price and volume adjustment according to trade/quote qualifier summarization actions to historical time series data.

Normally, the back-end should strictly serve what clients need. However, if the back-end cannot support them, the back-end can still return the form that the back-end supports with the proper adjustments in the response and status block (if applicable) instead of an error message.

Limitations: Adjustment behaviors listed in the limitation section may be changed or improved in the future.

1. If any combination of correction types is specified (i.e., exchangeCorrection or manualCorrection), all correction types will be applied to data in applicable event types.

2. If any combination of CORAX is specified (i.e., CCH, CRE, RPO, and RTS), all CORAX will be applied to data in applicable event types.

Adjustments values for Interday-summaries and Intraday-summaries API

If unspecified, each back-end service will be controlled with the proper adjustments in the response so that the clients know which adjustment types are applied by default. In this case, the returned data will be applied with exchange and manual corrections and applied with CORAX adjustments.

If specified, the clients want to get some specific adjustment types applied or even unadjusted.


Notes:

1. Summaries data will always have exchangeCorrection and manualCorrection applied. If the request is explicitly asked for uncorrected data, a status block will be returned along with the corrected data saying, "Uncorrected summaries are currently not supported".

2. The unadjusted will be ignored when other values are specified.

Below is a sample code to retrieve Daily historical pricing for the RIC defined in ricName variable, and I will set the start date from 2020 to nowaday. You need to set the interval to rdp.Intervals.DAILY to get intraday data. We will display the columns name with the heads and tails of the dataframe to review the data.


In [ ]:
import pandas as pd
import datetime
from IPython.display import clear_output

pd.set_option('future.no_silent_downcasting', True)

ricName='MSFT.O'
StartDate = '2020.01.01'
EndDate = str(datetime.date.today())

response = historical_pricing.summaries.Definition(
    universe = ricName,
    interval = Intervals.DAILY, # Supported intervals: DAILY, WEEKLY, MONTHLY, QUARTERLY, YEARLY.
    start = StartDate,
    end = EndDate).get_data()

if response is not None:
    #Show information of the dataframe to see column name , null and null-null field and the data type of the column
    response.data.df.info()
    display(response.data.df.head)
    display(response.data.df.tail)
else:
    print("Error while process the data")



# Preparing data for plotting chart

Next steps, we will create a new dataframe object from the data returned by the interday-summaries endpoint. We need to map the columns from the original dataframe to Open, High, Low, Close(OHLC) data. So we will define column names to map the OHLC data from the following variables.

```
openColumns =>Open 
highColumns => High
lowColumns => Low 
closeColumns => Close
volumeColums => Volume
```

In [ ]:
openColumns="OPEN_PRC"
highColumns="HIGH_1"
lowColumns="LOW_1"
closeColumns="TRDPRC_1"
volumeColumns="ACVOL_UNS"

ohlc_dataframe= response.data.df[[openColumns,highColumns,lowColumns,closeColumns,volumeColumns]]
ohlc_dataframe.index.name = 'Date'
display(ohlc_dataframe)
#rename the column name
ohlc_dataframe.info()
ohlc_dataframe=ohlc_dataframe.rename(columns={openColumns:"Open",highColumns:"High",lowColumns:"Low",closeColumns:"Close",volumeColumns:"Volume"})
print(ohlc_dataframe.columns)
display(ohlc_dataframe)


## Visualizing Stock Data

## Plot a simple Daily Closing Price line graph

We can create a simple line graph to compare open and close price using the following codes. You can change size of figure.figsize to adjust chart size.


In [ ]:
import matplotlib.pyplot as plt

%matplotlib ipympl

fig = plt.figure(figsize=(9,8),dpi=100)
ax = plt.subplot2grid((3,3), (0, 0), rowspan=3, colspan=3)
titlename='Graph for '+ricName
ax.set_title(titlename)
ax.set_xlabel('Date')
ax.set_ylabel('Open Price')
ax.grid(True)
ax.plot(ohlc_dataframe.index, ohlc_dataframe.Open) 
plt.show()

### Generate a Histogram of the Daily Closing Price

A histogram might be useful to help review daily closing prices over time to see the spread or volatility, and also the type of distribution. We use the seaborn __histplot__ method to plot the graph.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(9,8),dpi=100)
dfPlot=ohlc_dataframe[['Close','Volume']].loc['2020-01-01':datetime.date.today(),:]
graph=sns.histplot(dfPlot.Close.dropna(), 
                   bins=50, 
                   color='green',
                   kde=True,
                   stat="density", 
                   kde_kws=dict(cut=3))
graph.set(title=ricName)
plt.show()

## Using mplfiance to generate CandleStick and OHLC chart.

Next step we will generate a CandleStick and OHLC chart using the new version [mplfinance library](https://github.com/matplotlib/mplfinance). We need to pass a dataframe which contain Open, High, Low and Close data to the mplfinance.plot function and specifiy the name of the chart you want to plot. 

In [ ]:
import mplfinance as mpf
import datetime
%matplotlib ipympl
ohlc_dataframe.info()
#Volume use Int64 it will returns fails when mplfinance.plot check the support type of the data.So we need to convert it to Int64 and remove na from the data frame.
#If you don't care about Volume you can drop it from the dataframe.
dfPlot=ohlc_dataframe.dropna().astype({col: 'int64' for col in ohlc_dataframe.select_dtypes('Int64').columns}).loc['2024-01-01':str(datetime.date.today())]     
display(dfPlot)
dfPlot.info()
mpf.plot(dfPlot.loc[:str(datetime.date.today()),:],type='candle',style='charles',volume=True)
#mpf.plt(dfPlot,type='lines',volume=True)

Display shorter period.

In [ ]:
tempPlot= ohlc_dataframe.dropna().astype({col: 'int64' for col in ohlc_dataframe.select_dtypes('Int64').columns}).loc['2025-01-01':str(datetime.date.today())] 
mpf.plot(tempPlot,type='candle',style='charles',volume=True)

From a candlestick chart(zoom the graph), a green candlestick indicates a day where the closing price was higher than the open(Gain), while a red candlestick indicates a day where the open was higher than the close (Loss). The wicks indicate the high and the low, and the body the open and close (hue is used to determine which end of the body is open and which the close). You can follow the instruction from the following [example](https://github.com/matplotlib/mplfinance/blob/master/examples/styles.ipynb) to change the color by passing you own stytle to the plot function. And as I said previously, a user can use Candlestick charts for technical analysis and use them to make trading decisions, depending on the shape, color, and position of the candles. We will not cover a technical analysis in this example.

## Plot OHLC chart

An OHLC chart is a type of bar chart that shows open, high, low, and closing prices for each period. OHLC charts are useful since they show the four major data points over a period, with the closing price being considered the most important by many traders.The chart type is useful because it can show increasing or decreasing momentum. When the open and close are far apart it shows strong momentum, and when the open and close are close together it shows indecision or weak momentum. The high and low show the full price range of the period, useful in assessing volatility. There several patterns traders watch for on OHLC charts[[8]](https://www.investopedia.com/terms/o/ohlcchart.asp).

To plot OHLC chart you can just change the type to 'ohlc'. It's quite easy when using mplfinance.

In [ ]:
mpf.plot(dfPlot.loc['2024-01-01':str(datetime.date.today()),:],type='ohlc',style='charles',volume=True)

In [ ]:
display(dfPlot)
mpf.plot(dfPlot.loc['2025-01-01':str(datetime.date.today()),:],type='ohlc',style='charles',volume=True)

## Adding plots to the basic mplfinance plot()

Sometimes you may want to plot additional data within the same figure as the basic OHLC or Candlestick plot. For example, you may want to add the results of a technical study or some additional market data.

This is done by passing information into the call to mplfinance.plot() using the addplot ("additional plot") keyword.
I will show you a sample of the additional plots by adding a line plot for the data from columns High and Low to the original OHLC chart.

In [ ]:
dfSubPlot = dfPlot.loc['2024-12-01':str(datetime.date.today()),:]
apdict = mpf.make_addplot(dfSubPlot[['High','Low']])
mpf.plot(dfSubPlot,type='ohlc',style='charles',volume=True,addplot=apdict)

## Add Simple Moving Average to the graph

Next steps, we will add a moving average (MA) to the CandleStick chart. MA is a widely used indicator in technical analysis that helps smooth out price action by filtering out the “noise” from random short-term price fluctuations. It is a trend-following or lagging, indicator because it is based on past prices. The two basic and commonly used moving averages are the simple moving average (SMA), which is the simple average of a security over a defined number of time periods, and the exponential moving average (EMA), which gives greater weight to more recent prices. Note that this example will use only builtin MA provided by the mplfinance. The most common applications of moving averages are to identify the trend direction and to determine support and resistance levels. 

Basically, mplfinance provides functionality for easily computing a moving averages. The following codes creating a 20-day moving average from the price provided in the the data frame, and plotting it alongside the stock. Moving averages lag behind current price action because they are based on past prices; the longer the time period for the moving average, the greater the lag. Thus, a 200-day MA will have a much greater degree of lag than a 20-day MA because it contains prices for the past 200 days.



In [ ]:
mpf.plot(dfPlot.loc[:str(datetime.date.today()),:],type='candle',style='charles',mav=(20),volume=True)

The length of the moving average to use depends on the trading objectives, with shorter moving averages used for short-term trading and longer-term moving averages more suited for long-term investors. The 50-day and 200-day MAs are widely followed by investors and traders, with breaks above and below this moving average considered to be important trading signals.

The following codes use to generated CandleStick chart with multiple periods of times for SMA (20-day,50-day,75-day, and 200-day).

In [ ]:
mpf.plot(dfPlot.loc[:str(datetime.date.today()),:],type='candle',style='charles',mav=(20,60,75,200),volume=True)

You can also calculate the moving average from your own method or algorithm using the original data from the dataframe and then add a subplot to the CandleStick charts by using add plot like the previous sample of OHLC charts.

## Generate Renko Chart

Based on information from [the investopedia page](https://www.investopedia.com/terms/r/renkochart.asp), a Renko chart is a type of chart, developed by the Japanese, that is built using price movement rather than both price and standardized time intervals like most charts are. It is thought to be named after the Japanese word for bricks, "renga," since the chart looks like a series of bricks. A new brick is created when the price moves a specified price amount, and each block is positioned at a 45-degree angle (up or down) to the prior brick. An up brick is typically colored white or green, while a down brick is typically colored black or red[[7]](https://www.investopedia.com/terms/r/renkochart.asp).

Renko charts filter out noise and help traders to more clearly see the trend, since all movements that are smaller than the box size are filtered out.Renko charts typically only use closing prices based on the chart time frame chosen. For example, if using a weekly time frame, then weekly closing prices will be used to construct the bricks[[7]](https://www.investopedia.com/terms/r/renkochart.asp).

In [ ]:
mpf.plot(dfPlot,type='renko',style='charles',renko_params=dict(brick_size=4))



## Summary

This article explains users' alternate choices to retrieve such kind of the End of Day price or intraday datatime series data from the LSEG Data Libaray for Python. This article provides a sample code to use the LSEG Data Library for python to retrieve the Historical Pricing service, and then shows how to utilize the data with the 3rd party library such as the mplfinance to plot charts like the CandleStick, OHLC, and Renko chart for stock price technical analysis. With the library, users can specify a different kind of interval and adjustment behavior to retrieve more specific data and visualize the data on various charts. They can then use the charts to identify trading opportunities in price trends and patterns seen on charts. 

## References

1) [Data Platform](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis)

2) [mplfinance Library](https://github.com/matplotlib/mplfinance)

3) [CandleStick chart what is it?](https://www.investopedia.com/trading/candlestick-charting-what-is-it/)

4) [What Is a Moving Average Article.](https://www.investopedia.com/terms/m/movingaverage.asp)

5) [Seborn Tutorial](https://seaborn.pydata.org/tutorial/distributions.html)

6) [Matplotlib Examples](https://matplotlib.org/examples/pylab_examples/finance_demo.html)

7) [Renko Chart Definition and Uses](https://www.investopedia.com/terms/r/renkochart.asp)

8) [OHLC Chart](https://www.investopedia.com/terms/o/ohlcchart.asp)